In [1]:
import json

# Load the original JSON data
with open('../Dataset/train.json', 'r') as json_file:
    data = json.load(json_file)

# Initialize lists for labels and paths
filtered_labels = []
filtered_paths = []
filtered_data = []
filtered_ids = []

# Filter the data based on SENTENCE_DURATION between 2 and 3 seconds
for item in data:
    if 2.0 <= item['SENTENCE_DURATION'] <= 3.0:
        filtered_data.append(item)
        filtered_labels.append("<bos> " + item['SENTENCE_DESCRIPTION'] + " <eos>")  # Assuming 'SENTENCE_DESCRIPTION' contains labels
        filtered_paths.append(f"../Dataset/train/features/{item['SENTENCE_NAME']}.npy")
        filtered_ids.append(item['SENTENCE_NAME'])

filtered_data = filtered_data[0:100]

print(len(filtered_data))

# Write the filtered labels and paths to separate JSON files
with open('train.json', 'w') as labels_file:
    json.dump(filtered_data, labels_file, indent=4)

100


In [2]:
from mlflow import MLFlow

input_json_file = './train.json'
output_folder = './train/'
# processing_steps = ["crop", "reduce_noise", "add_noise", "rotate", "brightness", "contrast", "saturation"]

processor = MLFlow(input_json_file, output_folder)
vectors_df = processor.process_video_and_extract_features()

1/1 [==============================] - 0s 306ms/step
./train/features\--8pSDeC-fg_1-5-rgb_front
1/1 [==============================] - 0s 304ms/step
./train/features\--8pSDeC-fg_11-5-rgb_front
./train/features\--8pSDeC-fg_5-5-rgb_front
./train/features\--8pSDeC-fg_7-5-rgb_front
./train/features\--dANj_01AU_10-5-rgb_front
./train/features\--dANj_01AU_11-5-rgb_front
./train/features\--dANj_01AU_16-5-rgb_front
./train/features\--dANj_01AU_19-5-rgb_front
1/1 [==============================] - 0s 311ms/step
./train/features\--dANj_01AU_5-5-rgb_front
./train/features\-06_nJnhORg_19-5-rgb_front
1/1 [==============================] - 0s 307ms/step
./train/features\-06_nJnhORg_9-5-rgb_front
./train/features\-0FA6OhxZSE_11-5-rgb_front
./train/features\-0JdYlTf9Y8_11-5-rgb_front
./train/features\-0JdYlTf9Y8_3-5-rgb_front
./train/features\-0ZqnVFv7eI_0-5-rgb_front
./train/features\-1KpdDGPCq4_1-5-rgb_front
1/1 [==============================] - 0s 300ms/step
./train/features\-1PeVdIwiq0_1-8-rgb_fr

In [ ]:
import os
import json

videoId = []
videoSeq = []
videoSeq_raw = []
videoId_raw = []
features = []

In [ ]:
for i in range(0, 100):
    if os.path.exists(filtered_paths[i]):
        features.append(filtered_paths[i])
        videoId_raw.append(filtered_ids[i])
        videoSeq_raw.append(filtered_labels[i])

In [ ]:
import numpy as np

x_data = {}
for i in range(0, len(features)):
    f = np.load(features[i])
    f = f.reshape(-1, int(np.prod(f.shape[1:])))
    if f.shape[1] == 487872:
        print(f.shape)
        x_data[features[i][26:-4]] = f
        videoId.append(videoId_raw[i])
        videoSeq.append(videoSeq_raw[i])

print(len(x_data))
print(len(videoId))
print(len(videoSeq))

In [ ]:
print(videoSeq[1], videoId[1])

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_decoder_tokens = 195

tokenizer = Tokenizer(num_words=num_decoder_tokens)
tokenizer.fit_on_texts(videoSeq)
word_index = tokenizer.word_index   
print ('Convert to index sequences.')
train_sequences = tokenizer.texts_to_sequences(videoSeq)
train_sequences = pad_sequences(train_sequences, padding='post',truncating='post')
train_sequences = np.array(train_sequences)
train_sequences = pad_sequences(train_sequences, padding='post',truncating='post')
print(train_sequences.shape)
max_seq_length = train_sequences.shape[1]
filesize = len(train_sequences)

In [ ]:
print(filesize)

In [ ]:
from keras.utils import to_categorical


encoder_input_data = []
decoder_input_data = []
decoder_target_data = []
X_data = []
y_data = []
vCount = 0
curFilename = videoId[0]
for idx in range(0,filesize):
    if(videoId[idx] == curFilename):
        vCount = vCount + 1
        if(vCount > 2):
            continue
        else:
            vCount = 1
            curFilename = videoId[idx]
        encoder_input_data.append(x_data[videoId[idx]])
        y = to_categorical(train_sequences[idx], num_decoder_tokens)
        decoder_input_data.append(y[:-1])
        decoder_target_data.append(y[1:])
    encoder_input_data = np.array(encoder_input_data)
    decoder_input_data = np.array(decoder_input_data)
    decoder_target_data = np.array(decoder_target_data)

time_steps_decoder = decoder_input_data.shape[1]

In [ ]:
from keras.layers import Input, LSTM, Dense, Permute, Reshape, Activation, Flatten

time_steps_encoder = None
num_encoder_tokens = 487872
latent_dim = 32

encoder_inputs = Input(shape=(time_steps_encoder, num_encoder_tokens), name="encoder_inputs")
encoder = LSTM(latent_dim, return_state=True,return_sequences=True, name='endcoder_lstm')
_, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(time_steps_decoder, num_decoder_tokens), name= "decoder_inputs")
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='relu', name='decoder_relu')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
from keras.models import Model

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

In [ ]:
model.compile(metrics=['accuracy'], optimizer='adam', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=32, epochs=10)

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
encoder_model.summary()
decoder_model.summary()

In [ ]:
encoder_model.save(os.path.join('./', 'encoder_model.h5'))
decoder_model.save_weights(os.path.join('./', 'decoder_model_weights.h5'))

In [ ]:
from keras.models import Model, load_model

inf_encoder_model = load_model(os.path.join('./', 'encoder_model.h5'))
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
inf_decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
inf_decoder_model.load_weights(os.path.join('./', 'decoder_model_weights.h5'))

In [ ]:
import pickle, functools, operator

def decode_sequence2bs(input_seq):
    states_value = inf_encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, tokenizer.word_index['bos']] = 1
    beam_search(target_seq, states_value,[],[],0)
    return decode_seq

def beam_search(target_seq, states_value, prob,  path, lens):
    global decode_seq
    max_propablity = -1
    node = 2
    output_tokens, h, c = inf_decoder_model.predict(
        [target_seq] + states_value)
    output_tokens = output_tokens.reshape((num_decoder_tokens))
    sampled_token_index = output_tokens.argsort()[-node:][::-1]
    states_value = [h, c]
    for i in range(node):
        if sampled_token_index[i] == 0:
            sampled_char = ''
        else:
            print(tokenizer.word_index.values())
            sampled_char = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(sampled_token_index[i])]
        MAX_LEN = 9
        if(sampled_char != 'eos' and lens <= MAX_LEN):
            p = output_tokens[sampled_token_index[i]]
            if(sampled_char == ''):
                p = 1
            prob_new = list(prob)
            prob_new.append(p)
            path_new = list(path)
            path_new.append(sampled_char)
            target_seq = np.zeros((1, 1, num_decoder_tokens))
            target_seq[0, 0, sampled_token_index[i]] = 1.
            beam_search(target_seq, states_value, prob_new, path_new, lens+1)
        else:
            p = output_tokens[sampled_token_index[i]]
            prob_new = list(prob)
            prob_new.append(p)
            p = functools.reduce(operator.mul, prob_new, 1)
            if(p > max_propablity):
                decode_seq = path
                max_propablity = p

def decoded_sentence_tuning(decoded_sentence):
    decode_str = []
    filter_string = ['bos', 'eos']
    unigram = {}
    last_string = ""
    for idx2, c in enumerate(decoded_sentence):
        if c in unigram:
            unigram[c] += 1
        else:
            unigram[c] = 1
        if(last_string == c and idx2 > 0):
            continue
        if c in filter_string:
            continue
        if len(c) > 0:
            decode_str.append(c)
        if idx2 > 0:
            last_string = c
    return decode_str

In [ ]:
X_test = []
X_test_filename = []

max_propablity = -1

f = np.load(os.path.join('../Dataset/train/features/--8pSDeC-fg_11-5-rgb_front.npy'))
# f = f.reshape(-1, int(np.prod(f.shape[1:])))
# print(f.shape)
X_test.append(f)
X_test_filename.append('--8pSDeC-fg_11-5-rgb_front')

with open(os.path.join('./', 'test_output.txt'), 'w') as file:
    for idx, x in enumerate(X_test): 
        file.write(X_test_filename[idx]+',')
        decoded_sentence = decode_sequence2bs(x.reshape(-1, 60, 487872))
        decode_str = decoded_sentence_tuning(decoded_sentence)
        for d in decode_str:
            file.write(d + ' ')
        file.write('\n')
        # re-init max prob
        max_propablity = -1

In [ ]:
print("Predicted Sentence: " + X_test_filename[idx] + ':' + str(decode_str))
print("Actual Sentence: " + X_test_filename[idx] + ':' + videoSeq[1])
